<script type="text/javascript"
  src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=TeX-AMS_CHTML">
</script>
<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    tex2jax: {
      inlineMath: [['$','$'], ['\\(','\\)']],
      processEscapes: true},
      jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
      extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
      TeX: {
      extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
      equationNumbers: {
      autoNumber: "AMS"
      }
    }
  });
</script>

Jakub Musiał

# **AOD - lab2**

## **Exercise 4 - Containers**

Knowing the container arrangement and the cameras' field of view, determine the optimal camera arrangement (minimal number of cameras to cover all containers)

In [1]:
using JuMP
using GLPK

import JSON

#### **Data and utils**

In [2]:
# Data initialization
data_general = JSON.parse(read("data.json", String))
# Extraxt exercise 1 data from the general dictionary
data = data_general["ex4"]

Dict{String, Any} with 4 entries:
  "m"          => 5
  "k"          => 2
  "containers" => Dict{String, Any}("c1"=>Any[1, 2], "c9"=>Any[5, 4], "c2"=>Any…
  "n"          => 5

In [3]:
m = data["m"]
n = data["n"]
k = data["k"]
containers = sort(collect(keys(data["containers"])))
m, n, k, containers

(5, 5, 2, ["c0", "c1", "c10", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9"])

In [4]:
cx(container::String) = data["containers"][container][1]
cy(container::String) = data["containers"][container][2]

cy (generic function with 1 method)

#### **Build the model**

Notation:
* $C = \text{containers}$
* $[n] = \{1, ..., n\} \text{ : } n \epsilon \mathbf{N}$

In [5]:
model = Model()
set_optimizer(model, GLPK.Optimizer)

**Predictor variables:**

$x_{i, j}$ where $(i, j) \text{ } \epsilon \text{ } [m] \times [n]$ - indicator that there is a camera on the position $(i, j)$ 

In [6]:
@variable(model, x[range(1, m), range(1, n)], Bin)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 1:5
And data, a 5×5 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]
 x[5,1]  x[5,2]  x[5,3]  x[5,4]  x[5,5]

**Constraints:** 

* Indicators must be binary
  
  $(\forall{(i, j) \text{ } \epsilon \text{ } [m] \times [n]})(x_{i, j} \epsilon \{0, 1\})$

* Cameras cannot be placed where containers are located 
  
  $(\forall{c \epsilon C})(x_{c_x, c_y} = 0)$
  
* Each container must be seen by at least one camera
  
  $(\forall{c \epsilon C})(\sum_{(i, j) \text{ } \epsilon \text{ } \text{seen\_from}(c)} x_{i, j} \geq 1)$ 
  
  Where $\text{seen\_from}(c) = \{(c_x \pm i, c_y \pm i) \text{ : } i \epsilon [k]\} \text{ } \cap \text{ } ([m] \times [n])$

In [7]:
adjacent_dict = Dict{String, Any}()
for c in containers
    adjacent_dict[c] = []
    for i in range(1, k)
        x_ = cx(c)
        y_ = cy(c)
        # println(x_, y_)

        if (x_ - i > 0)
            push!(adjacent_dict[c], [x_ - i, y_])
        end
        if (x_ + i <= m)
            push!(adjacent_dict[c], [x_ + i, y_])
        end
        if (y_ - i > 0)
            push!(adjacent_dict[c], [x_, y_ - i])
        end
        if (y_ + i <= n)
            push!(adjacent_dict[c], [x_, y_ + i])
        end
    end
end

@constraints(model, begin
    [c in containers], x[cx(c), cy(c)] == 0
    [(c, adjacent) in adjacent_dict], sum(x[adj[1], adj[2]] for adj in adjacent) >= 1
end)

(1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["c0", "c1", "c10", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9"]
And data, a 11-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1,1] == 0.0
 x[1,2] == 0.0
 x[5,5] == 0.0
 x[1,4] == 0.0
 x[2,1] == 0.0
 x[2,5] == 0.0
 x[3,3] == 0.0
 x[3,5] == 0.0
 x[4,3] == 0.0
 x[5,3] == 0.0
 x[5,4] == 0.0, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, Pair{String, Any}["c1" => Any[[2, 2], [1, 1], [1, 3], [3, 2], [1, 4]], "c9" => Any[[4, 4], [5, 3], [5, 5], [3, 4], [5, 2]], "c2" => Any[

**Objective:** minimum number of cameras to cover all container locations

$min(\sum_{(i, j) \text{ } \epsilon \text{ } [m] \times [n]} x_{i, j})$

In [8]:
@objective(model, Min, sum(x[i, j] for i in range(1, m), j in range(1, n)))

x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] + x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[2,5] + x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] + x[4,1] + x[4,2] + x[4,3] + x[4,4] + x[4,5] + x[5,1] + x[5,2] + x[5,3] + x[5,4] + x[5,5]

In [9]:
# Optimize the model
optimize!(model)
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 4.00000e+00
  Objective bound    : 4.00000e+00
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 2.00009e-03


#### **Results:**

In [10]:
field = [[0 for _ in range(1, m)] for _ in range(1, n)]
for c in containers
    field[cx(c)][cy(c)] = 1
end

no_cameras = 0
for i in range(1, m), j in range(1, n)
    if (value(x[i, j]) == 1)
        println(" -> (", i, ", ", j, ")")
        field[i][j] = 2
    end
end

println("\nField:")
for i in range(1, m)
    for j in range(1, m)
        print(field[i][j], " ")
    end
    println()
end

 -> (1, 3)
 -> (3, 1)
 -> (4, 5)
 -> (5, 2)

Field:
1 1 2 1 0 
1 0 0 0 1 
2 0 1 0 1 
0 0 1 0 2 
0 2 1 1 1 


* Cameras arrangement for `k = k + 1`

In [11]:
k = k + 1

# Build the model
model = Model()
set_optimizer(model, GLPK.Optimizer)

# Predictor variables
@variable(model, x[range(1, m), range(1, n)], Bin)

# Constraints
adjacent_dict = Dict{String, Any}()
for c in containers
    adjacent_dict[c] = []
    for i in range(1, k)
        x_ = cx(c)
        y_ = cy(c)
        # println(x_, y_)

        if (x_ - i > 0)
            push!(adjacent_dict[c], [x_ - i, y_])
        end
        if (x_ + i <= m)
            push!(adjacent_dict[c], [x_ + i, y_])
        end
        if (y_ - i > 0)
            push!(adjacent_dict[c], [x_, y_ - i])
        end
        if (y_ + i <= n)
            push!(adjacent_dict[c], [x_, y_ + i])
        end
    end
end

@constraints(model, begin
    [c in containers], x[cx(c), cy(c)] == 0
    [(c, adjacent) in adjacent_dict], sum(x[adj[1], adj[2]] for adj in adjacent) >= 1
end)

# Objective
@objective(model, Min, sum(x[i, j] for i in range(1, m), j in range(1, n)))

# Optimize the model
optimize!(model)
solution_summary(model)

# Show results
field = [[0 for _ in range(1, m)] for _ in range(1, n)]
for c in containers
    field[cx(c)][cy(c)] = 1
end

println("Cameras:")
for i in range(1, m), j in range(1, n)
    if (value(x[i, j]) == 1)
        println(" -> (", i, ", ", j, ")")
        field[i][j] = 2
    end
end

println("\nField:")
for i in range(1, m)
    for j in range(1, m)
        print(field[i][j], " ")
    end
    println()
end

Cameras:
 -> (1, 3)
 -> (4, 5)
 -> (5, 1)

Field:
1 1 2 1 0 
1 0 0 0 1 
0 0 1 0 1 
0 0 1 0 2 
2 0 1 1 1 
